<a href="https://colab.research.google.com/github/kodai-utsunomiya/memorization-and-generalization/blob/main/numerical_experiments/1_time_scale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $(k, d)$-Sparse Parity Task

$d$-個の数字の内の $k$ 個の数字のパリティを計算する（$k \le d$）という問題

# データセット

---

- $\mathcal{D}_{k, d} = \{ (\boldsymbol{x}_i , y_i) \}_{i=1}^n$
    - $n$ 個の学習データ
    - $\boldsymbol{x}_i \in \{ 0,1 \}^d$：バイナリーベクトル．$\boldsymbol{x}_i \sim \text{Unif} \left( \{0,1\}^d \right)$
    - $y_i = \left(\sum_{i}^k x^{(i)} \right) \text{mod} \hspace{2mm} 2$ ：最初の $k$  個の数字（clean digits）のパリティ

<br>

  - $\boldsymbol{x}_i$ の残りの $d-k$ 個の数字（noisy digits）は $y_i$ とは無関係

<br>

- 例：
    - $(3, 30)$-sparse parity dataset
        - $\boldsymbol{x}_1$：<font color="blue">000</font>$110010110001010111001001011$，  $y_1 = 0$
        - $\boldsymbol{x}_2$：<font color="blue">010</font>$110010110001010111001001011$，  $y_2 = 1$
            
            $\hspace{2mm} \vdots$
            

<br>

- 学習データをまとめて，$\mathcal{X} = \left[ \boldsymbol{x}_1, \ldots, \boldsymbol{x}_n \right] \in \mathbb{R}^{n \times d}$，$\mathcal{Y} = \left[ y_1 \ldots, y_n \right] \in \mathbb{R}^n$ と行列表記

In [249]:
import numpy as np
import torch
from torch.utils.data import Dataset, Subset

def _generate_unique_binary_strings(n, total_size, data_seed):
    """
    ユニークなバイナリ文字列を生成する関数

    Parameters:
    - n: バイナリ文字列の長さ
    - total_size: 必要なユニークなバイナリ文字列の数
    - data_seed: ランダムシード

    Returns:
    - list: ユニークなバイナリ文字列のリスト．各バイナリ文字列は長さ n のタプル
    """
    np.random.seed(data_seed)
    unique_binary_strings = set()

    # 必要な数のユニークなバイナリ文字列が得られるまで繰り返す
    while len(unique_binary_strings) < total_size:
        binary_string = tuple(np.random.randint(2, size=n))
        unique_binary_strings.add(binary_string)
    return list(unique_binary_strings)

def _prepare_data(binary_strings, k):
    """
    入力データと出力ラベルを準備する関数

    Parameters:
    - binary_strings: バイナリ文字列のリスト．各バイナリ文字列は長さ n のタプル
    - k: 出力ラベルの計算に使用する最初の k 個のビット

    Returns:
    - tuple: (入力データ, 出力ラベル)
      - 入力データ: バイナリ文字列を NumPy 配列として保持し，最後にバイアス列を追加
      - 出力ラベル: 最初の k ビットの合計の 2 で割った余りとして計算
    """
    # バイナリ文字列をNumPy配列に変換
    inputs = np.array(binary_strings, dtype=np.float32)

    # 出力ラベルを計算 (最初の k ビットの合計を 2 で割った余り)
    outputs = np.sum(inputs[:, :k], axis=-1) % 2

    # 出力ラベルを 0 -> -1, 1 -> 1 に変換（ヒンジ損失用）
    outputs = 2 * outputs - 1

    return inputs, outputs

def _normalize_data(inputs):
    """
    データの中心化と標準化を行うヘルパー関数

    Parameters:
    - inputs: 正規化対象の入力データ (NumPy配列)

    Returns:
    - 正規化された入力データ
    """
    # NumPy配列をPyTorchテンソルに変換
    inputs = torch.tensor(inputs, dtype=torch.float32)

    # 1. データの中心化（平均を0に）
    inputs = inputs - inputs.mean(dim=0)

    # 2. 標準化（各サンプルのノルムで割る）．データが単位ノルムを持つようにスケーリングされる
    norm = inputs.flatten(1).norm(dim=1).view(-1, *(1,) * (inputs.dim() - 1))
    inputs = (inputs[0].numel() ** 0.5) * inputs / norm

    return inputs.numpy()  # NumPy配列に戻す

def get_binary_dataset(n, k, train_size, test_size, data_seed, normalize=False, device=None):
    """
    二値分類用にデータセットを取得する関数

    Parameters:
    - n: バイナリ文字列の長さ
    - k: 出力ラベルの計算に使用する最初の k 個のビット
    - train_size: 訓練データのサイズ
    - test_size: テストデータのサイズ
    - data_seed: ランダムシード
    - normalize: データを正規化するかどうかを示すフラグ
    - device: データを移動するデバイス

    Returns:
    - tuple: ((訓練データのx, 訓練データのy), (テストデータのx, テストデータのy))
      - 訓練データのx: 訓練データの入力データ
      - 訓練データのy: 訓練データの出力ラベル
      - テストデータのx: テストデータの入力データ
      - テストデータのy: テストデータの出力ラベル
    """
    total_size = train_size + test_size

    # ユニークなバイナリ文字列を生成
    binary_strings = _generate_unique_binary_strings(n, total_size, data_seed)

    # 入力データと出力ラベルを準備
    inputs, outputs = _prepare_data(binary_strings, k)

    # 正規化を行う場合（中心化と標準化）
    if normalize:
        inputs = _normalize_data(inputs)

    # データのインデックスをシャッフル
    indices = np.random.permutation(total_size)

    # 訓練データとテストデータのインデックスを分割
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    # 訓練データとテストデータをNumPy配列からPyTorchテンソルに変換
    train_inputs = torch.tensor(inputs[train_indices], dtype=torch.float32)
    train_outputs = torch.tensor(outputs[train_indices], dtype=torch.float32)
    test_inputs = torch.tensor(inputs[test_indices], dtype=torch.float32)
    test_outputs = torch.tensor(outputs[test_indices], dtype=torch.float32)

    # データを指定されたデバイスに移動
    if device:
        train_inputs, train_outputs = train_inputs.to(device), train_outputs.to(device)
        test_inputs, test_outputs = test_inputs.to(device), test_outputs.to(device)

    return (train_inputs, train_outputs), (test_inputs, test_outputs)

In [224]:
a = get_binary_dataset(n=30, k=3, train_size=900, test_size=100, data_seed=42, normalize=False)

In [243]:
print(f"訓練データのパターン\n", a[0][0].shape)
print(a[0][0])

訓練データのパターン
 torch.Size([900, 30])
tensor([[1., 0., 1.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 1., 1., 1.],
        [0., 1., 1.,  ..., 0., 1., 0.],
        ...,
        [1., 1., 1.,  ..., 1., 0., 1.],
        [0., 1., 1.,  ..., 0., 0., 1.],
        [1., 0., 0.,  ..., 1., 0., 1.]])


In [244]:
print(f"訓練データのラベル\n", a[0][1].shape)
print(a[0][1])

訓練データのラベル
 torch.Size([900])
tensor([-1.,  1., -1., -1., -1.,  1.,  1.,  1., -1., -1., -1., -1.,  1., -1.,
        -1., -1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.,
        -1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,
        -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,
        -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1., -1.,  1.,  1.,
         1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1.,  1.,
         1.,  1., -1., -1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,
        -1., -1.,  1.,  1.,  1., -1., -1., -1., -1., -1.,  1.,  1., -1.,  1.,
        -1.,  1.,  1., -1., -1.,  1., -1., -1., -1.,  1., -1.,  1., -1., -1.,
         1.,  1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1.,
         1.,  1., -1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,
        -1., -1., -1., -1., -1.,  1

In [231]:
a[1][0].shape

torch.Size([100, 30])

In [232]:
a[1][1].shape

torch.Size([100])

# モデル

---

$ F(\boldsymbol{w}, \boldsymbol{x}) \equiv \alpha \left\lbrack f(\boldsymbol{w}, \boldsymbol{x}) - f(\boldsymbol{w}_0, \boldsymbol{x}) \right\rbrack $ という形をしたモデルの学習を考える（これを予測器として使用し，$\boldsymbol{w}$を学習）

# FCクラス．<font color="green"> $f(\boldsymbol{x})$ の定義 </font>

### ネットワークの構造

1. **入力層**: 次元数 $d$ の入力を受け取る．
2. **隠れ層**: 層数 $L$ の隠れ層があり，各隠れ層のユニット数は $h$．
3. **出力層**: 最終層は出力がスカラー値である 1 次元のベクトルを生成．

<br>

### 層ごとの計算

1. **初期化**:
   - 隠れ層 $i$ の重み行列 $W_i$ は，次のように初期化：
     
     $
     W_i \sim \mathcal{N}(0, 1)
     $

     ここで，$W_i$ のサイズは $ h \times \text{hh}_{i}$ ．
     
     $\text{hh}_{i} $ は前の層の出力ユニット数．

   - メモリ効率を考慮し，重み行列を分割：
     
     \begin{aligned} W_i =  \begin{bmatrix}
        W_i^{(0)} \\
        W_i^{(1)} \\
        \vdots \\
        W_i^{(n-1)}
        \end{bmatrix}  \end{aligned}
     
     各部分行列 $W_i^{(j)}$ はサイズ $m \times \text{hh}_{i}$．ここで，$m$ は分割サイズ．

<br>

2. **順伝播計算**:
   - 入力テンソル $x$ は，初期の隠れ層で次のように変換：
     
     $
     x^{(0)} = x W_0^T / \sqrt{d}
     $

     ここで，$W_0$ は最初の隠れ層の重み行列．バイアス項がある場合，次のように加算：
     
     $
     x^{(0)} = x^{(0)} + b_0
     $

     その後，活性化関数 $ \sigma $ を適用：
     
     $
     x^{(1)} = \sigma(x^{(0)})
     $

   - 次の隠れ層も同様に計算．一般的に，隠れ層 $i$ の計算は次のようになる：
     
     $
     x^{(i)} = x^{(i-1)} W_i^T / \sqrt{h}
     $

     ここで，$W_i$ は現在の層の重み行列．バイアス項がある場合，次のように加算：

     $
     x^{(i)} = x^{(i)} + b_i
     $

     そして，活性化関数を適用：
     
     $
     x^{(i+1)} = \sigma(x^{(i)})
     $

   - 最終層では，次のように計算：
     
     $
     x^{(L)} = x^{(L-1)} W_L^T / \sqrt{h} + b_L
     $
     
     ここで，$W_L$ は最終層の重み行列．出力テンソル $x$ を 1 次元に変換して返す：

     $
     x^{(L)} = x^{(L)} \text{view}(-1)
     $

In [248]:
import functools
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

"""
全結合ネットワーク（Fully Connected Network, FC）のクラスを定義．
任意の層数 L を持ち，各層のユニット数は h で指定．
活性化関数 act は任意に指定可能で，バイアス項の有無も指定可能．
"""

class FC(nn.Module):
    def __init__(self, d, h, L, act, bias=False):
        super().__init__()

        # ネットワークの初期化
        hh = d  # 入力の次元数
        for i in range(L):
            # 隠れ層の重み行列を正規分布で初期化
            W = torch.randn(h, hh)

            # メモリ効率を考慮し，重み行列を部分行列に分割して ParameterList に格納
            # next two line are here to avoid memory issue when computing the kerne
            n = max(1, 128 * 256 // hh)  # 分割サイズを計算
            W = nn.ParameterList([nn.Parameter(W[j: j+n]) for j in range(0, len(W), n)])

            # 分割した重み行列をレイヤーとして登録
            setattr(self, "W{}".format(i), W)

            # バイアス項が指定されている場合は，それをゼロで初期化して登録
            if bias:
                self.register_parameter("B{}".format(i), nn.Parameter(torch.zeros(h)))

            # 次のレイヤーの入力次元は現在の隠れ層のユニット数になる
            hh = h

        # 最終層の重み行列を初期化（出力がスカラー値なので次元は (1, h)）
        self.register_parameter("W{}".format(L), nn.Parameter(torch.randn(1, hh)))

        # バイアス項が指定されている場合は，最終層のバイアスをゼロで初期化
        if bias:
            self.register_parameter("B{}".format(L), nn.Parameter(torch.zeros(1)))

        # クラス変数としてレイヤー数，活性化関数，バイアスの有無を保持
        self.L = L
        self.act = act
        self.bias = bias

    def forward(self, x):
        # 順伝播計算
        for i in range(self.L + 1):
            # i 番目の層の重み行列を取得
            W = getattr(self, "W{}".format(i))

            # ParameterList 形式の重み行列をフルの行列に結合
            if isinstance(W, nn.ParameterList):
                W = torch.cat(list(W))

            # バイアス項が指定されている場合は，バイアスを取得
            if self.bias:
                B = self.bias * getattr(self, "B{}".format(i))
            else:
                B = 0

            # 現在の入力の次元数を取得
            h = x.size(1)

            if i < self.L:
                # 隠れ層での線形変換とスケーリング，そして活性化関数の適用
                x = x @ (W.t() / h ** 0.5)  # 重み行列との積（次元スケーリング）
                x = self.act(x + B)  # バイアス項を加えた後，活性化関数を適用
            else:
                # 最終層での線形変換（出力はスカラー値）
                x = x @ (W.t() / h ** 0.5) + B  # スカラー出力

        # 出力を 1 次元のテンソルに変換して返す
        return x.view(-1)

In [191]:
b = FC(d=10, h=100, L=2, act=nn.Sigmoid(), bias=False)

In [192]:
b.forward(x=torch.randn(5, 10))

tensor([-0.5326, -0.5812, -0.5613, -0.5328, -0.5460], grad_fn=<ViewBackward0>)

In [203]:
b.W0[0].shape

torch.Size([100, 10])

# ダイナミクス

In [247]:
"""
Dynamics that compares the angle of the gradient between steps and keep it small

- マージンに達したときに停止

2つの実装：
1. `train_regular` - 任意のモデルに対応
2. `train_kernel` - 線形モデル専用
"""

import copy
import itertools
import math
from time import perf_counter
import torch


def gradient(outputs, inputs, grad_outputs=None, retain_graph=None, create_graph=False):
    '''
    `outputs` に対する `inputs` の勾配を計算する関数
    使用例:
    ```
    gradient(x.sum(), x)          # x の合計に対する勾配
    gradient((x * y).sum(), [x, y])  # x と y の要素ごとの積の合計に対する勾配
    ```

    :param outputs: 勾配を計算する対象の出力テンソル
    :param inputs: 勾配を計算したい入力テンソルのリストまたは単一テンソル
    :param grad_outputs: 出力テンソルの勾配を指定するためのオプション（通常は None で良い）
    :param retain_graph: 計算グラフを保持するかどうかを指定するフラグ（デフォルトは None）
    :param create_graph: 勾配の計算グラフを作成するかどうかを指定するフラグ（デフォルトは False）
    :return: 入力テンソルに対する勾配をフラットなテンソルとして返す
    '''

    # `inputs` がテンソルの場合はリストに変換
    if torch.is_tensor(inputs):
        inputs = [inputs]
    else:
        inputs = list(inputs)

    # `torch.autograd.grad` 関数を使用して勾配を計算
    grads = torch.autograd.grad(outputs, inputs, grad_outputs,
                                allow_unused=True, # 計算に使用されないテンソルには勾配が計算されない
                                retain_graph=retain_graph, # 計算グラフを保持するかどうか
                                create_graph=create_graph) # 勾配の計算グラフを作成するかどうか

    # 勾配が None の場合は，同じサイズのゼロテンソルを代わりに使用
    grads = [x if x is not None else torch.zeros_like(y) for x, y in zip(grads, inputs)]

    # 勾配テンソルをフラットな形状に変換して連結
    return torch.cat([x.contiguous().view(-1) for x in grads])


def loglinspace(rate, step, end=None):
    """
    対数線形間隔での数値を生成するジェネレーター関数
    対数的に変化する間隔で数値を生成

    `rate` と `step` のパラメータを使って，新しい値を計算
    `end` が指定されていない場合は無限に数値を生成

    Arguments:
        rate (float): 対数的な変化の速度を制御するパラメータ
        step (float): 各ステップでの間隔の大きさ
        end (float, optional): 生成を停止する条件となる最大値．指定されない場合は無限に生成

    Yields:
        float: 現在の時間 `t` の値を生成
    """
    t = 0
    while end is None or t <= end:
        yield t  # 現在の時間 `t` の値を生成
        # 次の `t` を計算．ここで，`math.exp(-t * rate / step)` は指数関数的な減衰を表す
        t = int(t + 1 + step * (1 - math.exp(-t * rate / step)))


class ContinuousMomentum(torch.optim.Optimizer):
    """連続的なモーメンタムを実装

    - d/dt velocity = -1/tau (velocity + grad)
    - または
    - d/dt velocity = -mu/t (velocity + grad)

    - d/dt parameters = velocity
    """

    def __init__(self, params, dt, tau):
        """
        初期化メソッド

        Arguments:
            params (iterable): 最適化するパラメータのリスト
            dt (float): 時間ステップのサイズ
            tau (float): モーメンタムのタイムコンスタント
        """
        defaults = dict(dt=dt, tau=tau)
        super().__init__(params, defaults)

    def step(self, closure=None):
        """単一の最適化ステップを実行

        Arguments:
            closure (callable, optional): モデルを再評価し，損失を返すクロージャ．多くの最適化器にはオプショナル．

        Returns:
            loss (Tensor or None): 損失の値．クロージャが指定された場合はその損失値を返す．
        """
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:
            tau = group['tau']
            dt = group['dt']

            for p in group['params']:
                # 勾配がないパラメータはスキップ
                if p.grad is None:
                    continue

                param_state = self.state[p]
                # パラメータの状態が初めてのときは，時間 t を 0 に設定
                if 't' not in param_state:
                    t = param_state['t'] = 0
                else:
                    t = param_state['t']

                # モーメンタムの状態（速度）を初期化
                if tau != 0:
                    if 'velocity' not in param_state:
                        v = param_state['velocity'] = torch.zeros_like(p.data)
                    else:
                        v = param_state['velocity']

                # モーメンタムの計算
                if tau > 0:
                    # tau > 0 の場合の連続モーメンタムの計算
                    x = math.exp(-dt / tau)  # 時間の経過とともに減衰する係数
                    v.mul_(x).add_(-(1 - x), p.grad.data) # 速度に勾配を加える
                elif tau < 0:
                    # tau < 0 の場合の連続モーメンタムの計算
                    mu = -tau
                    x = (t / (t + dt)) ** mu  # 時間の経過に伴う係数
                    v.mul_(x).add_(-(1 - x), p.grad.data)  # 速度に勾配を加える
                else:
                    # tau = 0 の場合のシンプルな勾配降下
                    v = -p.grad.data

                # パラメータの更新
                p.data.add_(dt, v)    # パラメータに速度を加える
                param_state['t'] += dt    # 時間を進める

        return loss


def make_step(f, optimizer, dt, grad):
    """
    指定された勾配 `grad` を使用して，最適化ステップを実行

    Arguments:
        f (torch.nn.Module): トレーニングするモデル
        optimizer (torch.optim.Optimizer): 使用する最適化器
        dt (float): 時間刻み
        grad (torch.Tensor): 勾配テンソル
    """
    i = 0
    # モデルの全パラメータに対してループ
    for p in f.parameters():
        # パラメータの総要素数を取得
        n = p.numel()
        # 勾配テンソルを対応するパラメータに合わせてリシェイプし，割り当て
        p.grad = grad[i: i + n].view_as(p)
        i += n  # インデックスを次のパラメータに進める

    # 各パラメータグループに対して時間刻み `dt` を設定
    for param_group in optimizer.param_groups:
        param_group['dt'] = dt

    # 最適化ステップを実行
    optimizer.step()

    # 勾配をリセット（計算グラフから切り離し）
    for p in f.parameters():
        p.grad = None


def train_regular(f0, x, y, tau, max_walltime, alpha, loss, subf0, max_dgrad=math.inf, max_dout=math.inf):
    """
    一般的なモデルのトレーニング関数

    Arguments:
        f0 (torch.nn.Module): 初期モデル
        x (torch.Tensor): 入力データ
        y (torch.Tensor): 出力ラベル
        tau (float): モーメンタムパラメータ
        max_walltime (float): 最大経過時間
        alpha (float): 出力変化に対する閾値
        loss (callable): ロス関数
        subf0 (bool): 初期モデルの出力を使用するかどうか
        max_dgrad (float): 勾配の変化の最大許容値
        max_dout (float): 出力の変化の最大許容値
    """

    # 初期モデルのコピーを作成
    f = copy.deepcopy(f0)

    # モデルの出力を計算（必要に応じて初期モデルの出力を使用）
    with torch.no_grad():
        out0 = f0(x) if subf0 else 0

    # 時間刻みとモーメンタムのパラメータを初期化
    dt = 1
    step_change_dt = 0

    # ContinuousMomentum オプティマイザを初期化
    optimizer = ContinuousMomentum(f.parameters(), dt=dt, tau=tau)

    # ログ-線形間隔のチェックポイント生成器を作成
    checkpoint_generator = loglinspace(0.01, 100)
    checkpoint = next(checkpoint_generator)

    # 経過時間を計測するためのタイマーを開始
    wall = perf_counter()
    t = 0
    converged = False

    # 最初のモデルの出力と勾配を計算
    out = f(x)
    grad = gradient(loss((out - out0) * y).mean(), f.parameters())

    # トレーニングループ
    for step in itertools.count():
        # 現在のモデルとオプティマイザの状態を保存
        state = copy.deepcopy((f.state_dict(), optimizer.state_dict(), t))

        while True:
            # 最適化ステップを実行
            make_step(f, optimizer, dt, grad)
            t += dt
            current_dt = dt

            # モデルの新しい出力と勾配を計算
            new_out = f(x)
            new_grad = gradient(loss((new_out - out0) * y).mean(), f.parameters())

            # 出力の変化量を計算
            dout = (out - new_out).mul(alpha).abs().max().item()

            # 勾配の変化量を計算
            if grad.norm() == 0 or new_grad.norm() == 0:
                dgrad = 0
            else:
                dgrad = (grad - new_grad).norm().pow(2).div(grad.norm() * new_grad.norm()).item()

            # 勾配の変化量と出力の変化量が閾値以下であれば，時間刻みを増加
            if dgrad < max_dgrad and dout < max_dout:
                if dgrad < 0.5 * max_dgrad and dout < 0.5 * max_dout:
                    dt *= 1.1
                break
            # そうでない場合は，時間刻みを減少
            dt /= 10

            # 現在の状態を出力
            print("[{} +{}] [dt={:.1e} dgrad={:.1e} dout={:.1e}]".format(step, step - step_change_dt, dt, dgrad, dout), flush=True)

            # モデルとオプティマイザの状態をリストア
            step_change_dt = step
            f.load_state_dict(state[0])
            optimizer.load_state_dict(state[1])
            t = state[2]

        # 新しい出力と勾配を保存
        out = new_out
        grad = new_grad

        save = False

        # チェックポイントに達した場合，または収束した場合に状態を保存
        if step == checkpoint:
            checkpoint = next(checkpoint_generator)
            assert checkpoint > step
            save = True

        # 出力が閾値を超え，収束していない場合に収束を宣言
        if (alpha * (out - out0) * y >= 1).all() and not converged:
            converged = True
            save = True

        if save:
            state = {
                'step': step,
                'wall': perf_counter() - wall,
                't': t,
                'dt': current_dt,
                'dgrad': dgrad,
                'dout': dout,
                'norm': sum(p.norm().pow(2) for p in f.parameters()).sqrt().item(),
                'dnorm': sum((p0 - p).norm().pow(2) for p0, p in zip(f0.parameters(), f.parameters())).sqrt().item(),
                'grad_norm': grad.norm().item(),
            }

            yield f, state, converged

        # 収束した場合はトレーニングを終了
        if converged:
            break

        # 経過時間が最大経過時間を超えた場合にトレーニングを終了
        if perf_counter() > wall + max_walltime:
            break

        # 出力に NaN が含まれている場合，トレーニングを終了
        if torch.isnan(out).any():
            break



def train_kernel(ktrtr, ytr, tau, max_walltime, alpha, loss_prim, max_dgrad=math.inf, max_dout=math.inf):
    """
    線形モデル専用のトレーニング関数

    Arguments:
        ktrtr (torch.Tensor): カーネル行列
        ytr (torch.Tensor): 出力ラベル
        tau (float): モーメンタムパラメータ
        max_walltime (float): 最大経過時間
        alpha (float): 出力変化に対する閾値
        loss_prim (callable): プライムロス関数
        max_dgrad (float): 勾配の変化の最大許容値
        max_dout (float): 出力の変化の最大許容値
    """
    # 初期出力と速度ベクトルをゼロで初期化
    otr = ktrtr.new_zeros(len(ytr))
    velo = otr.clone()

    # 時間刻みとステップ変更のタイミングを初期化
    dt = 1
    step_change_dt = 0

    # ログ-線形間隔のチェックポイント生成器を作成
    checkpoint_generator = loglinspace(0.01, 100)
    checkpoint = next(checkpoint_generator)

    # 経過時間を計測するためのタイマーを開始
    wall = perf_counter()
    t = 0
    converged = False

    # 初期のプライムロスを計算
    lprim = loss_prim(otr * ytr) * ytr
    grad = ktrtr @ lprim / len(ytr)

    # トレーニングループ
    for step in itertools.count():

        # 現在の出力，速度，時間を保存
        state = copy.deepcopy((otr, velo, t))

        while True:
            # モーメンタムパラメータに基づいて速度を更新
            if tau > 0:
                x = math.exp(-dt / tau)
                velo.mul_(x).add_(-(1 - x), grad)
            elif tau < 0:
                mu = -tau
                x = (t / (t + dt)) ** mu
                velo.mul_(x).add_(-(1 - x), grad)
            else:
                velo.copy_(-grad)

            # 出力を更新
            otr.add_(dt, velo)

            t += dt
            current_dt = dt

            # 新しいプライムロスを計算し，勾配を更新
            lprim = loss_prim(otr * ytr) * ytr
            new_grad = ktrtr @ lprim / len(ytr)

            # 出力の変化量を計算
            dout = velo.mul(dt * alpha).abs().max().item()

            # 勾配の変化量を計算
            if grad.norm() == 0 or new_grad.norm() == 0:
                dgrad = 0
            else:
                dgrad = (grad - new_grad).norm().pow(2).div(grad.norm() * new_grad.norm()).item()

            # 勾配と出力の変化量が許容範囲内であれば，時間刻みを増加
            if dgrad < max_dgrad and dout < max_dout:
                if dgrad < 0.1 * max_dgrad and dout < 0.1 * max_dout:
                    dt *= 1.1
                break

            # そうでない場合は，時間刻みを減少
            dt /= 10

            # 現在の状態を出力
            print("[{} +{}] [dt={:.1e} dgrad={:.1e} dout={:.1e}]".format(step, step - step_change_dt, dt, dgrad, dout), flush=True)
            step_change_dt = step

            # 保存した状態をリストア
            otr.copy_(state[0])
            velo.copy_(state[1])
            t = state[2]

        # 勾配を新しい値で更新
        grad = new_grad

        save = False

        # チェックポイントに達した場合，または収束した場合に状態を保存
        if step == checkpoint:
            checkpoint = next(checkpoint_generator)
            assert checkpoint > step
            save = True

        # 出力が閾値を超え，収束していない場合に収束を宣言
        if (alpha * otr * ytr >= 1).all() and not converged:
            converged = True
            save = True

        if save:
            state = {
                'step': step,
                'wall': perf_counter() - wall,
                't': t,
                'dt': current_dt,
                'dgrad': dgrad,
                'dout': dout,
                'grad_norm': grad.norm().item(),
            }

            yield otr, velo, grad, state, converged

        # 収束した場合はトレーニングを終了
        if converged:
            break

        # 経過時間が最大経過時間を超えた場合にトレーニングを終了
        if perf_counter() > wall + max_walltime:
            break

        # 出力に NaN が含まれている場合，トレーニングを終了
        if torch.isnan(otr).any():
            break

# カーネル

`compute_kernels` 関数は，与えられたモデル $ f $ の入力データ $ x_{\text{tr}} $ と $ x_{\text{te}} $ に基づいて，カーネル行列（Gram 行列）を計算．

- $ K_{\text{trtr}} $：トレーニングデータ間のカーネル行列
- $ K_{\text{tetr}} $：テストデータとトレーニングデータ間のカーネル行列
- $ K_{\text{tete}} $：テストデータ間のカーネル行列

これらの行列は，モデルのパラメータに関する勾配を使って計算．

<br>

### モデルの勾配計算

モデル $ f $ が入力 $ x $ に対して出力を生成し，その勾配を計算．勾配の計算は以下のように行う：

1. 入力 $ x $ に対するモデルの出力を $ f(x) $ とする．
2. この出力に対するパラメータ $ \theta $ の勾配を求める：$ \nabla_\theta f(x) $

  ここで，$ \nabla_\theta f(x) $ は $ x $ に対する勾配であり，モデルのパラメータ $ \theta $ に関する勾配ベクトル．

<br>

### Gram 行列の計算

1. $ K_{\text{trtr}} $

  トレーニングデータ $ x_{\text{tr}} $ に対するカーネル行列 $ K_{\text{trtr}} $ は，各トレーニングデータポイント $ x_i $ と $ x_j $ の勾配ベクトルに基づいて計算：

  $ K_{\text{trtr}} = J_{\text{tr}} J_{\text{tr}}^T $

  ここで，$ J_{\text{tr}} $ はトレーニングデータ $ x_{\text{tr}} $ に対する勾配ベクトルを列に持つ行列．$ J_{\text{tr}} $ の $ i $-th 行は，入力 $ x_i $ に対する勾配ベクトル．

<br>

2. $ K_{\text{tetr}} $

  テストデータ $ x_{\text{te}} $ とトレーニングデータ $ x_{\text{tr}} $ とのカーネル行列 $ K_{\text{tetr}} $ は次のように計算：

  $ K_{\text{tetr}} = J_{\text{te}} J_{\text{tr}}^T $

  ここで，$ J_{\text{te}} $ はテストデータ $ x_{\text{te}} $ に対する勾配ベクトルを列に持つ行列．

<br>

3. $ K_{\text{tete}} $

  テストデータ $ x_{\text{te}} $ に対するカーネル行列 $ K_{\text{tete}} $ は、テストデータポイント $ x_i $ と $ x_j $ の勾配ベクトルに基づいて計算：

  $ K_{\text{tete}} = J_{\text{te}} J_{\text{te}}^T $

  ここで，$ J_{\text{te}} $ の $ i $-th 行は，入力 $ x_i $ に対する勾配ベクトル．

In [246]:
"""
Computes the Gram matrix of a given model
"""

def compute_kernels(f, xtr, xte):
    # from hessian import gradient

    # 新しいゼロ行列を作成
    # ktrtr: トレーニングデータ間のカーネル行列
    # ktetr: テストデータとトレーニングデータ間のカーネル行列
    # ktete: テストデータ間のカーネル行列
    ktrtr = xtr.new_zeros(len(xtr), len(xtr))
    ktetr = xtr.new_zeros(len(xte), len(xtr))
    ktete = xtr.new_zeros(len(xte), len(xte))

    params = []
    current = []

    # モデルのパラメータをサイズで降順にソートし，メモリ制限に基づいて分割
    for p in sorted(f.parameters(), key=lambda p: p.numel(), reverse=True):
        current.append(p)
        # メモリ制限に基づき，パラメータを分割
        if sum(p.numel() for p in current) > 2e9 // (8 * (len(xtr) + len(xte))):
            if len(current) > 1:
                params.append(current[:-1])
                current = current[-1:]
            else:
                params.append(current)
                current = []
    if len(current) > 0:
        params.append(current)

    # 各パラメータグループについてカーネル行列を計算
    for i, p in enumerate(params):
        print("[{}/{}] [len={} numel={}]".format(i, len(params), len(p), sum(x.numel() for x in p)), flush=True)

        # 勾配行列を初期化
        jtr = xtr.new_empty(len(xtr), sum(u.numel() for u in p))  # (P, N~) # (トレーニングデータ数, パラメータ数の合計)
        jte = xte.new_empty(len(xte), sum(u.numel() for u in p))  # (P, N~) # (テストデータ数, パラメータ数の合計)

        # トレーニングデータに対する勾配行列を計算
        for j, x in enumerate(xtr):
            jtr[j] = gradient(f(x[None]), p)  # (N~) # (パラメータ数の合計)

        # テストデータに対する勾配行列を計算
        for j, x in enumerate(xte):
            jte[j] = gradient(f(x[None]), p)  # (N~) # (パラメータ数の合計)

        # カーネル行列を更新
        ktrtr.add_(jtr @ jtr.t())  # トレーニングデータ間のカーネル行列
        ktetr.add_(jte @ jtr.t())  # テストデータとトレーニングデータ間のカーネル行列
        ktete.add_(jte @ jte.t())  # テストデータ間のカーネル行列
        del jtr, jte  # 不要になった勾配行列を削除

    return ktrtr, ktetr, ktete

# 損失関連

In [250]:
from functools import partial

def loss_func(hyper, fy):
    """
    損失関数を計算．指定された損失関数のタイプに応じて異なる計算を行う．

    Parameters:
    - hyper: 設定を含むオブジェクト
    - fy: モデルの予測値と実際のラベルを用いた計算結果

    Returns:
    - 損失値: 指定された損失関数に基づいて計算された損失
    """
    if hyper.loss == 'softhinge':
        sp = partial(torch.nn.functional.softplus, beta=hyper.lossbeta)
        # 損失を計算し，alpha で正規化
        return sp(1 - hyper.alpha * fy) / hyper.alpha


def loss_func_prime(hyper, fy):
    """
    損失関数の勾配（微分）を計算．指定された損失関数のタイプに応じて異なる計算を行う．

    Parameters:
    - hyper: 設定を含むオブジェクト
    - fy: モデルの予測値と実際のラベルを用いた計算結果

    Returns:
    - 損失の勾配: 損失関数の微分
    """
    if hyper.loss == 'softhinge':
        return -torch.sigmoid(hyper.lossbeta * (1 - hyper.alpha * fy)).mul(hyper.lossbeta)

# run_kernel と run_regular

In [245]:
def run_kernel(hyper, ktrtr, ktetr, ktete, f, xtr, ytr, xte, yte):
    """
    カーネル回帰の実行と評価を行う関数．トレーニングデータに対するカーネル回帰を実行し，トレーニングとテストデータに対する出力を計算

    Parameters:
    - hyper: 設定やパラメータが格納されたオブジェクト
    - ktrtr: トレーニングデータに対するカーネル行列
    - ktetr: トレーニングデータとテストデータ間のカーネル行列
    - ktete: テストデータに対するカーネル行列
    - f: モデル関数
    - xtr: トレーニングデータの入力
    - ytr: トレーニングデータのラベル
    - xte: テストデータの入力
    - yte: テストデータのラベル

    Returns:
    - out: トレーニングおよびテストの結果とカーネル行列の統計量を含む辞書
    """

    # 引数の f0 が 1 であることを確認
    assert hyper.f0 == 1

    # 結果を保存するためのリストを初期化
    dynamics = []

    # 時間スケール tau を設定
    tau = hyper.tau_over_h * hyper.h
    # alpha に基づいて tau を調整
    if hyper.tau_alpha_crit is not None:
        tau *= min(1, hyper.tau_alpha_crit / hyper.alpha)

    # カーネル回帰のトレーニングを実行し，各ステップの状態を取得
    for otr, _velo, _grad, state, _converged in train_kernel(ktrtr, ytr, tau, hyper.train_time, hyper.alpha, partial(loss_func_prime, hyper), hyper.max_dgrad, hyper.max_dout):

        # トレーニングの結果を状態に保存
        state['train'] = {
            'loss': loss_func(hyper, otr * ytr).mean().item(),
            'aloss': hyper.alpha * loss_func(hyper, otr * ytr).mean().item(),
            'err': (otr * ytr <= 0).double().mean().item(),
            'nd': (hyper.alpha * otr * ytr < 1).long().sum().item(),
            'dfnorm': otr.pow(2).mean().sqrt(),
            'outputs': otr if hyper.save_outputs else None,
            'labels': ytr if hyper.save_outputs else None,
        }

        print("[i={d[step]:d} t={d[t]:.2e} wall={d[wall]:.0f}] [dt={d[dt]:.1e} dgrad={d[dgrad]:.1e} dout={d[dout]:.1e}] [train aL={d[train][aloss]:.2e} err={d[train][err]:.2f} nd={d[train][nd]}]".format(d=state), flush=True)
        dynamics.append(state)

    # トレーニングデータに対するカーネル回帰の係数を計算
    c = torch.lstsq(otr.view(-1, 1), ktrtr).solution.flatten()

    # テストデータの出力を計算
    if len(xte) > len(xtr):
        # from hessian import gradient
        # テストデータがトレーニングデータより多い場合，勾配を用いて出力を計算
        a = gradient(f(xtr) @ c, f.parameters())
        ote = torch.stack([gradient(f(x[None]), f.parameters()) @ a for x in xte])
    else:
        # それ以外の場合，事前に計算されたカーネル行列を使用
        ote = ktetr @ c

    # 結果を辞書としてまとめて返す
    out = {
        'dynamics': dynamics,
        'train': {
            'outputs': otr,
            'labels': ytr,
        },
        'test': {
            'outputs': ote,
            'labels': yte,
        },
        'kernel': {
            'train': {
                'value': ktrtr.cpu() if hyper.store_kernel == 1 else None,
                'diag': ktrtr.diag(),
                'mean': ktrtr.mean(),
                'std': ktrtr.std(),
                'norm': ktrtr.norm(),
            },
            'test': {
                'value': ktete.cpu() if hyper.store_kernel == 1 else None,
                'diag': ktete.diag(),
                'mean': ktete.mean(),
                'std': ktete.std(),
                'norm': ktete.norm(),
            },
        },
    }

    return out


def run_regular(hyper, f0, xtr, ytr, xte, yte):
    """
    通常のトレーニングプロセスを実行し，トレーニングとテストの結果を収集する関数

    Parameters:
    - hyper: 設定やパラメータが格納されたオブジェクト
    - f0: 初期モデル関数
    - xtr: トレーニングデータの入力
    - ytr: トレーニングデータのラベル
    - xte: テストデータの入力
    - yte: テストデータのラベル

    Yields:
    - f: トレーニング後のモデル関数
    - out: トレーニングとテストの結果を含む辞書
    """

    # 勾配計算を無効にし，初期モデルによる出力を計算
    with torch.no_grad():
        otr0 = f0(xtr)
        ote0 = f0(xte)

    # 初期モデルが無い場合は，出力をゼロに設定
    if hyper.f0 == 0:
        otr0 = torch.zeros_like(otr0)
        ote0 = torch.zeros_like(ote0)

    # ランダムにサンプルインデックスを選択し，部分データを取得
    j = torch.randperm(min(len(xte), len(xtr)))[:10 * hyper.chunk]
    ytrj = ytr[j]
    ytej = yte[j]

    # トレーニング開始時間を記録
    t = perf_counter()

    # 時間スケール tau を設定
    tau = hyper.tau_over_h * hyper.h
    if hyper.tau_alpha_crit is not None:
        tau *= min(1, hyper.tau_alpha_crit / hyper.alpha)

    # トレーニングのダイナミクスを保存するリスト
    dynamics = []

    # トレーニングを実行し，各ステップの状態を取得
    for f, state, done in train_regular(f0, xtr, ytr, tau, hyper.train_time, hyper.alpha, partial(loss_func, hyper), bool(hyper.f0), hyper.max_dgrad, hyper.max_dout):
        with torch.no_grad():
            # 勾配計算を無効にし，トレーニングとテストデータの出力を計算
            otr = f(xtr[j]) - otr0[j]
            ote = f(xte[j]) - ote0[j]

        # モデルアーキテクチャが 'fc' の場合，重みのノルムを計算
        if hyper.arch.split('_')[0] == 'fc':
            def getw(f, i):
                return torch.cat(list(getattr(f.f, "W{}".format(i))))
            state['wnorm'] = [getw(f, i).norm().item() for i in range(f.f.L + 1)]
            state['dwnorm'] = [(getw(f, i) - getw(f0, i)).norm().item() for i in range(f.f.L + 1)]

        # トレーニングデータの損失，エラー，その他の情報を計算
        state['train'] = {
            'loss': loss_func(hyper, otr * ytrj).mean().item(),
            'aloss': hyper.alpha * loss_func(hyper, otr * ytrj).mean().item(),
            'err': (otr * ytr[j] <= 0).double().mean().item(),
            'nd': (hyper.alpha * otr * ytr[j] < 1).long().sum().item(),
            'dfnorm': otr.pow(2).mean().sqrt(),
            'fnorm': (otr + otr0[j]).pow(2).mean().sqrt(),
            'outputs': otr if hyper.save_outputs else None,
            'labels': ytrj if hyper.save_outputs else None,
        }

        # テストデータの損失，エラー，その他の情報を計算
        state['test'] = {
            'loss': loss_func(hyper, ote * ytej).mean().item(),
            'aloss': hyper.alpha * loss_func(hyper, ote * ytej).mean().item(),
            'err': (ote * yte[j] <= 0).double().mean().item(),
            'nd': (hyper.alpha * ote * yte[j] < 1).long().sum().item(),
            'dfnorm': ote.pow(2).mean().sqrt(),
            'fnorm': (ote + ote0[j]).pow(2).mean().sqrt(),
            'outputs': ote if hyper.save_outputs else None,
            'labels': ytej if hyper.save_outputs else None,
        }

        print("[i={d[step]:d} t={d[t]:.2e} wall={d[wall]:.0f}] [dt={d[dt]:.1e} dgrad={d[dgrad]:.1e} dout={d[dout]:.1e}] [train aL={d[train][aloss]:.2e} err={d[train][err]:.2f} nd={d[train][nd]}/{p}] [test aL={d[test][aloss]:.2e} err={d[test][err]:.2f}]".format(d=state, p=len(j)), flush=True)

        # ダイナミクスリストに現在のステートを追加
        dynamics.append(state)

        # トレーニングが完了した場合または実行時間が 120秒 を超えた場合に結果を出力
        if done or perf_counter() - t > 120:
            t = perf_counter()

            # 勾配計算を無効にし，最終的なトレーニングとテストの出力を計算
            with torch.no_grad():
                otr = f(xtr) - otr0
                ote = f(xte) - ote0

            # 結果を辞書としてまとめて返す
            out = {
                'dynamics': dynamics,
                'train': {
                    'f0': otr0,
                    'outputs': otr,
                    'labels': ytr,
                },
                'test': {
                    'f0': ote0,
                    'outputs': ote,
                    'labels': yte,
                }
            }
            yield f, out

# メイン

In [267]:
import os
import torch

class SplitEval(torch.nn.Module):
    """
    モデルを分割して評価するためのクラス．入力データを指定されたサイズに分割し，それぞれの分割に対してモデルを評価．
    最後に，評価結果を結合して返す．

    Attributes:
    - f: 評価対象のモデル
    - size: 入力データを分割するサイズ
    """
    def __init__(self, f, size):
        """
        コンストラクタ．モデルと分割サイズを初期化

        Parameters:
        - f: 評価対象のモデル
        - size: 入力データを分割するサイズ
        """
        super().__init__()
        self.f = f    # モデルを保存
        self.size = size    # 分割サイズを保存

    def forward(self, x):
        """
        フォワードパス．入力データを指定されたサイズに分割し，各分割に対してモデルを適用し，結果を結合して返す．

        Parameters:
        - x: 入力データ（バッチ）

        Returns:
        - torch.Tensor: 各分割に対してモデルを適用した結果を結合したテンソル
        """

        # 入力データを指定されたサイズで分割し，各分割に対してモデル f を適用
        # 分割サイズに基づいてリスト内包表記を使用して結果を取得
        # それらの結果を torch.cat で結合して返す
        return torch.cat([self.f(x[i: i + self.size]) for i in range(0, len(x), self.size)])

def run_exp(hyper, f0, xtr, ytr, xte, yte):
    """
    実験を実行し，トレーニングとテストの結果を収集する関数

    Parameters:
    - hyper: 設定やパラメータが格納されたオブジェクト
    - f0: 初期モデル関数
    - xtr: トレーニングデータの入力
    - ytr: トレーニングデータのラベル
    - xte: テストデータの入力
    - yte: テストデータのラベル

    Yields:
    - run: 実験の結果を含む辞書
    """

    # 実験の結果を格納する辞書を初期化
    run = {
        'hyper': hyper,
        'N': sum(p.numel() for p in f0.parameters()),  # モデルのパラメータの総数
    }

    # delta_kernel または init_kernel が有効な場合，カーネルを計算
    if hyper.delta_kernel == 1 or hyper.init_kernel == 1:
        init_kernel = compute_kernels(f0, xtr, xte[:len(xtr)])

    # init_kernel が有効な場合，カーネルを計算して run 辞書に追加
    if hyper.init_kernel == 1:
        run['init_kernel'] = run_kernel(hyper, *init_kernel, f0, xtr, ytr, xte, yte)

    # delta_kernel が有効な場合，初期カーネルを CPU に移動し，init_kernel を削除
    if hyper.delta_kernel == 1:
        init_kernel = (init_kernel[0].cpu(), init_kernel[2].cpu())
    elif hyper.init_kernel == 1:
        del init_kernel

    # regular が有効な場合，トレーニングを実行し，結果を run 辞書に追加
    if hyper.regular == 1:
        for f, out in run_regular(hyper, f0, xtr, ytr, xte, yte):
            run['regular'] = out
            yield run

        # delta_kernel または final_kernel が有効な場合，最終カーネルを計算
        if hyper.delta_kernel == 1 or hyper.final_kernel == 1:
            final_kernel = compute_kernels(f, xtr, xte[:len(xtr)])

        # final_kernel が有効な場合，カーネルを計算して run 辞書に追加
        if hyper.final_kernel == 1:
            run['final_kernel'] = run_kernel(hyper, *final_kernel, f, xtr, ytr, xte, yte)

        # delta_kernel が有効な場合，初期カーネルと最終カーネルの差を計算し，run 辞書に追加
        if hyper.delta_kernel == 1:
            final_kernel = (final_kernel[0].cpu(), final_kernel[2].cpu())
            run['delta_kernel'] = {
                'train': (init_kernel[0] - final_kernel[0]).norm().item(),
                'test': (init_kernel[1] - final_kernel[1]).norm().item(),
            }

    # 実験の結果を yield して返す
    yield run

def execute(hyper):
    """
    実験を実行するためのメイン関数．データセットの準備，モデルの設定，実験の実行

    Parameters:
    - hyper: 実験の設定を含むオブジェクト
    """
    # CUDNN の最適化を有効にする（CUDA が利用可能な場合）
    torch.backends.cudnn.benchmark = True

    # CUDA が利用可能かどうかを確認し，デバイスを設定
    if torch.cuda.is_available():
        device = torch.device('cuda')
        # デフォルトのデータ型を設定
        if hyper.dtype == 'float64':
            torch.set_default_dtype(torch.float64)
        elif hyper.dtype == 'float32':
            torch.set_default_dtype(torch.float32)
    else:
        device = torch.device('cpu')
        print("CUDA is not available. Running on CPU.")
        # デフォルトのデータ型を設定
        if hyper.dtype == 'float64':
            torch.set_default_dtype(torch.float64)
        elif hyper.dtype == 'float32':
            torch.set_default_dtype(torch.float32)

    # データセットの準備
    (xtr, ytr), (xte, yte) = get_binary_dataset(hyper.n, hyper.k, hyper.train_size, hyper.test_size, hyper.data_seed, hyper.normalize, device)
    # データ型をデフォルトに設定
    xtr = xtr.type(torch.get_default_dtype())
    xte = xte.type(torch.get_default_dtype())
    ytr = ytr.type(torch.get_default_dtype())
    yte = yte.type(torch.get_default_dtype())

    # ランダムシードの設定
    torch.manual_seed(hyper.init_seed + hash(hyper.alpha))

    # アーキテクチャと活性化関数の設定
    arch, act = hyper.arch.split('_')
    if act == 'relu':
        act = lambda x: 2 ** 0.5 * torch.relu(x)
    elif act == 'tanh':
        act = torch.tanh
    elif act == 'softplus':
        # softplus のスケーリングファクターを計算
        factor = torch.nn.functional.softplus(torch.randn(100000, dtype=torch.float64), hyper.spbeta).pow(2).mean().rsqrt().item()
        act = lambda x: torch.nn.functional.softplus(x, beta=hyper.spbeta).mul(factor)
    else:
        raise ValueError('act not specified')

    # モデルの設定
    if arch == 'fc':
        assert hyper.L is not None
        xtr = xtr.flatten(1)
        xte = xte.flatten(1)
        # 全結合層のモデルを作成
        f = FC(xtr.size(1), hyper.h, hyper.L, act, hyper.bias).to(device)
    else:
        raise ValueError('arch not specified')

    # モデルを分割して評価するためのラッパーを適用
    f = SplitEval(f, hyper.chunk)

    # バッチシードの設定
    torch.manual_seed(hyper.batch_seed)

    # 実験の実行と結果の取得
    for run in run_exp(hyper, f, xtr, ytr, xte, yte):
        yield run

class HyperParams:
    def __init__(self):
        self.device = 'cuda'
        self.dtype = 'float64'
        self.init_seed = 0
        self.batch_seed = 0
        self.n = 30
        self.k = 3
        self.train_size = 700
        self.test_size = 1000
        self.normalize = False
        self.data_seed = 0
        self.alpha = 1e-4
        self.f0 = 1
        self.tau_over_h = 1e-3
        self.tau_alpha_crit = 1e3
        self.L = 3
        self.h = 50
        self.arch = 'fc_softplus'
        self.spbeta = 5
        self.bias = False
        self.max_dgrad = 1e-4
        self.max_dout = 0.1
        self.loss = 'softhinge'
        self.lossbeta = 20
        self.train_time = 18000
        self.chunk = 100
        self.init_kernel = 0
        self.delta_kernel = 0
        self.final_kernel = 0
        self.store_kernel = 0
        self.save_outputs = 0
        self.regular = 1
        self.directory = 'F10k3Lsp_h_init'
        self.pickle = 'F10k3Lsp_h_init.pickle'


# 実験の実行と結果を保存する関数
def run_and_save_experiment(hyper):
    # ディレクトリの作成
    if not os.path.exists(hyper.directory):
        os.makedirs(hyper.directory)

    # pickle ファイルのパスをディレクトリに基づいて変更
    pickle_path = os.path.join(hyper.directory, hyper.pickle)

    # test_size または chunk が None の場合は train_size の値で初期化
    if hyper.test_size is None:
        hyper.test_size = hyper.train_size

    if hyper.chunk is None:
        hyper.chunk = hyper.train_size

    # 引数を pickle ファイルに保存
    torch.save(hyper, pickle_path)

    try:
        for res in execute(hyper):
            # 結果を pickle ファイルに追加で保存
            with open(pickle_path, 'wb') as f:
                torch.save(hyper, f)
                torch.save(res, f)
    except Exception as e:
        os.remove(pickle_path)
        raise e


######## 実験の実行
hyper = HyperParams()
run_and_save_experiment(hyper)

CUDA is not available. Running on CPU.
[0 +0] [dt=1.0e-01 dgrad=1.5e-04 dout=3.3e-06]
[i=0 t=1.00e-01 wall=0] [dt=1.0e-01 dgrad=1.1e-06 dout=2.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=1 t=2.10e-01 wall=0] [dt=1.1e-01 dgrad=1.8e-06 dout=3.5e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=2 t=3.31e-01 wall=0] [dt=1.2e-01 dgrad=2.2e-06 dout=3.9e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=3 t=4.64e-01 wall=0] [dt=1.3e-01 dgrad=2.7e-06 dout=4.3e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=4 t=6.11e-01 wall=0] [dt=1.5e-01 dgrad=3.3e-06 dout=4.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=5 t=7.72e-01 wall=0] [dt=1.6e-01 dgrad=4.0e-06 dout=5.2e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=6 t=9.49e-01 wall=0] [dt=1.8e-01 dgrad=4.8e-06 dout=5.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 

In [268]:
import os

pickle_path = 'F10k3Lsp_h_init/F10k3Lsp_h_init.pickle'

if not os.path.exists(pickle_path):
    print("File does not exist.")
else:
    print(f"File size: {os.path.getsize(pickle_path)} bytes")

File size: 2284964 bytes


In [269]:
import os
import torch

def load_and_print_summary(pickle_path):
    # pickle ファイルが存在するか確認
    if not os.path.exists(pickle_path):
        raise FileNotFoundError(f"{pickle_path} does not exist")

    # pickle ファイルを読み込む
    try:
        with open(pickle_path, 'rb') as f:
            # 最初の読み込みで HyperParams オブジェクトを読み込む
            hyper = torch.load(f)
            # 次に実験結果を読み込む
            results = []
            while True:
                try:
                    results.append(torch.load(f))
                except EOFError:
                    break

        # HyperParams オブジェクトの内容を表示
        print("HyperParameters:")
        for key, value in vars(hyper).items():
            print(f"{key}: {value}")

        # 実験結果の内容を表示
        print("\nExperiment Results:")
        for result in results:
            print(result)
    except Exception as e:
        print(f"Error loading file: {e}")

# pickle ファイルのパスを指定
pickle_path = 'F10k3Lsp_h_init/F10k3Lsp_h_init.pickle'

# 読み込みと表示
load_and_print_summary(pickle_path)

Error loading file: PytorchStreamReader failed reading zip archive: invalid header or archive is corrupted


<ipython-input-269-1efd64ea2c9b>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hyper = torch.load(f)


In [276]:
import os
import torch

class SplitEval(torch.nn.Module):
    """
    モデルを分割して評価するためのクラス．入力データを指定されたサイズに分割し，それぞれの分割に対してモデルを評価．
    最後に，評価結果を結合して返す．

    Attributes:
    - f: 評価対象のモデル
    - size: 入力データを分割するサイズ
    """
    def __init__(self, f, size):
        """
        コンストラクタ．モデルと分割サイズを初期化

        Parameters:
        - f: 評価対象のモデル
        - size: 入力データを分割するサイズ
        """
        super().__init__()
        self.f = f    # モデルを保存
        self.size = size    # 分割サイズを保存

    def forward(self, x):
        """
        フォワードパス．入力データを指定されたサイズに分割し，各分割に対してモデルを適用し，結果を結合して返す．

        Parameters:
        - x: 入力データ（バッチ）

        Returns:
        - torch.Tensor: 各分割に対してモデルを適用した結果を結合したテンソル
        """
        return torch.cat([self.f(x[i: i + self.size]) for i in range(0, len(x), self.size)])

def run_exp(hyper, f0, xtr, ytr, xte, yte):
    """
    実験を実行し，トレーニングとテストの結果を収集する関数

    Parameters:
    - hyper: 設定やパラメータが格納されたオブジェクト
    - f0: 初期モデル関数
    - xtr: トレーニングデータの入力
    - ytr: トレーニングデータのラベル
    - xte: テストデータの入力
    - yte: テストデータのラベル

    Yields:
    - run: 実験の結果を含む辞書
    """
    run = {
        'hyper': hyper,
        'N': sum(p.numel() for p in f0.parameters()),  # モデルのパラメータの総数
    }

    if hyper.delta_kernel == 1 or hyper.init_kernel == 1:
        init_kernel = compute_kernels(f0, xtr, xte[:len(xtr)])

    if hyper.init_kernel == 1:
        run['init_kernel'] = run_kernel(hyper, *init_kernel, f0, xtr, ytr, xte, yte)

    if hyper.delta_kernel == 1:
        init_kernel = (init_kernel[0].cpu(), init_kernel[2].cpu())
    elif hyper.init_kernel == 1:
        del init_kernel

    if hyper.regular == 1:
        for f, out in run_regular(hyper, f0, xtr, ytr, xte, yte):
            run['regular'] = out
            yield run

        if hyper.delta_kernel == 1 or hyper.final_kernel == 1:
            final_kernel = compute_kernels(f, xtr, xte[:len(xtr)])

        if hyper.final_kernel == 1:
            run['final_kernel'] = run_kernel(hyper, *final_kernel, f, xtr, ytr, xte, yte)

        if hyper.delta_kernel == 1:
            final_kernel = (final_kernel[0].cpu(), final_kernel[2].cpu())
            run['delta_kernel'] = {
                'train': (init_kernel[0] - final_kernel[0]).norm().item(),
                'test': (init_kernel[1] - final_kernel[1]).norm().item(),
            }

    yield run

def execute(hyper):
    """
    実験を実行するためのメイン関数．データセットの準備，モデルの設定，実験の実行

    Parameters:
    - hyper: 実験の設定を含むオブジェクト
    """
    torch.backends.cudnn.benchmark = True

    if torch.cuda.is_available():
        device = torch.device('cuda')
        if hyper.dtype == 'float64':
            torch.set_default_dtype(torch.float64)
        elif hyper.dtype == 'float32':
            torch.set_default_dtype(torch.float32)
    else:
        device = torch.device('cpu')
        print("CUDA is not available. Running on CPU.")
        if hyper.dtype == 'float64':
            torch.set_default_dtype(torch.float64)
        elif hyper.dtype == 'float32':
            torch.set_default_dtype(torch.float32)

    (xtr, ytr), (xte, yte) = get_binary_dataset(hyper.n, hyper.k, hyper.train_size, hyper.test_size, hyper.data_seed, hyper.normalize, device)
    xtr = xtr.type(torch.get_default_dtype())
    xte = xte.type(torch.get_default_dtype())
    ytr = ytr.type(torch.get_default_dtype())
    yte = yte.type(torch.get_default_dtype())

    torch.manual_seed(hyper.init_seed + hash(hyper.alpha))

    arch, act = hyper.arch.split('_')
    if act == 'relu':
        act = lambda x: 2 ** 0.5 * torch.relu(x)
    elif act == 'tanh':
        act = torch.tanh
    elif act == 'softplus':
        factor = torch.nn.functional.softplus(torch.randn(100000, dtype=torch.float64), hyper.spbeta).pow(2).mean().rsqrt().item()
        act = lambda x: torch.nn.functional.softplus(x, beta=hyper.spbeta).mul(factor)
    else:
        raise ValueError('act not specified')

    if arch == 'fc':
        assert hyper.L is not None
        xtr = xtr.flatten(1)
        xte = xte.flatten(1)
        f = FC(xtr.size(1), hyper.h, hyper.L, act, hyper.bias).to(device)
    else:
        raise ValueError('arch not specified')

    f = SplitEval(f, hyper.chunk)

    torch.manual_seed(hyper.batch_seed)

    for run in run_exp(hyper, f, xtr, ytr, xte, yte):
        yield run

class HyperParams:
    def __init__(self):
        self.device = 'cuda'
        self.dtype = 'float64'
        self.init_seed = 0
        self.batch_seed = 0
        self.n = 30
        self.k = 3
        self.train_size = 700
        self.test_size = 1000
        self.normalize = False
        self.data_seed = 0
        self.alpha = 1e-4
        self.f0 = 1
        self.tau_over_h = 1e-3
        self.tau_alpha_crit = 1e3
        self.L = 3
        self.h = 50
        self.arch = 'fc_softplus'
        self.spbeta = 5
        self.bias = False
        self.max_dgrad = 1e-4
        self.max_dout = 0.1
        self.loss = 'softhinge'
        self.lossbeta = 20
        self.train_time = 18000
        self.chunk = 100
        self.init_kernel = 0
        self.delta_kernel = 0
        self.final_kernel = 0
        self.store_kernel = 0
        self.save_outputs = 0
        self.regular = 1
        self.directory = 'F10k3Lsp_h_init'
        self.pickle = 'F10k3Lsp_h_init.pickle'

# 実験の実行と結果を保存する関数
import pickle

def run_and_save_experiment(hyper):
    # ディレクトリの作成
    if not os.path.exists(hyper.directory):
        os.makedirs(hyper.directory)

    # pickle ファイルのパスをディレクトリに基づいて変更
    pickle_path = os.path.join(hyper.directory, hyper.pickle)

    # test_size または chunk が None の場合は train_size の値で初期化
    if hyper.test_size is None:
        hyper.test_size = hyper.train_size

    if hyper.chunk is None:
        hyper.chunk = hyper.train_size

    try:
        # 引数を pickle ファイルに保存
        with open(pickle_path, 'wb') as f:
            pickle.dump(hyper, f)

        with open(pickle_path, 'ab') as f:
            for res in execute(hyper):
                # 結果を pickle ファイルに追加で保存
                pickle.dump(res, f)
    except Exception as e:
        if os.path.exists(pickle_path):
            os.remove(pickle_path)
        print(f"An error occurred during saving: {e}")
        raise e

######## 実験の実行と結果の保存
hyper = HyperParams()
run_and_save_experiment(hyper)

CUDA is not available. Running on CPU.
[0 +0] [dt=1.0e-01 dgrad=1.5e-04 dout=3.3e-06]
[i=0 t=1.00e-01 wall=0] [dt=1.0e-01 dgrad=1.1e-06 dout=2.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=1 t=2.10e-01 wall=0] [dt=1.1e-01 dgrad=1.8e-06 dout=3.5e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=2 t=3.31e-01 wall=0] [dt=1.2e-01 dgrad=2.2e-06 dout=3.9e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=3 t=4.64e-01 wall=0] [dt=1.3e-01 dgrad=2.7e-06 dout=4.3e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=4 t=6.11e-01 wall=0] [dt=1.5e-01 dgrad=3.3e-06 dout=4.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=5 t=7.72e-01 wall=0] [dt=1.6e-01 dgrad=4.0e-06 dout=5.2e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 err=0.52]
[i=6 t=9.49e-01 wall=0] [dt=1.8e-01 dgrad=4.8e-06 dout=5.8e-07] [train aL=1.00e+00 err=0.48 nd=700/700] [test aL=1.00e+00 

<ipython-input-276-441e17659eda>:223: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  hyper = torch.load(f)


In [331]:
import pickle

# ANSI escape sequences for colors
COLORS = {
    'HEADER': '\033[95m',  # Magenta
    'OKCYAN': '\033[96m',  # Cyan
    'OKGREEN': '\033[92m',  # Green
    'WARNING': '\033[93m',  # Yellow
    'FAIL': '\033[91m',    # Red
    'ENDC': '\033[0m',     # Reset to default
    'BOLD': '\033[1m',     # Bold
    'UNDERLINE': '\033[4m' # Underline
}

def color_text(text, color_name):
    return f"{COLORS[color_name]}{text}{COLORS['ENDC']}"

def load_and_print_summary(file_path, verbose=False):
    """
    保存された pickle ファイルを読み込み，その中身の概要を出力する関数

    Parameters:
    - file_path: 読み込む pickle ファイルのパス
    - verbose: データの詳細表示を行うかどうかのフラグ
    """
    try:
        with open(file_path, 'rb') as f:
            # 初めに HyperParams オブジェクトを読み込む
            hyper = pickle.load(f)

            # ハイパーパラメータの概要表示
            print(color_text(f"Loaded HyperParams object:", 'HEADER'))
            print(f"  Type: {type(hyper).__name__}")
            print(f"  Attributes:")
            for attribute in dir(hyper):
                if not attribute.startswith('__') and not callable(getattr(hyper, attribute)):
                    value = getattr(hyper, attribute)
                    print(f"    {attribute}: {value} ({type(value).__name__})")

            # 次に、実験結果が続く場合
            results = []
            while True:
                try:
                    result = pickle.load(f)
                    results.append(result)
                except EOFError:
                    break

            # 実験結果の概要を表示
            print(color_text("\nExperiment results summary:", 'OKCYAN'))
            print(f"  Number of results: {len(results)}")
            if len(results) > 0:
                if isinstance(results[0], dict):
                    print("  Sample keys from result dictionaries:")
                    sample_result = results[0]
                    sample_keys = {}
                    if isinstance(sample_result, dict):
                        for key, value in sample_result.items():
                            if isinstance(value, (list, dict)):
                                entry_count = len(value)
                                sample_keys[key] = (f"{entry_count} entries", type(value).__name__)
                            else:
                                sample_keys[key] = (f"{value}", type(value).__name__)
                    print(f"    Number of keys: {len(sample_keys)}")
                    for key, (entry_or_value, dtype) in sample_keys.items():
                        print(f"    {key}: {entry_or_value}, Type: {dtype}")
                else:
                    print(f"  Type of results: {type(results[0]).__name__}")

            # `regular` に含まれるキーとその数を表示
            if len(results) > 0 and isinstance(results[0], dict):
                regular_keys = set()
                key_data_counts = {}

                for result in results:
                    if 'regular' in result and isinstance(result['regular'], dict):
                        for key, value in result['regular'].items():
                            if key not in key_data_counts:
                                key_data_counts[key] = (0, None)
                            count, dtype = key_data_counts[key]
                            key_data_counts[key] = (count + 1, dtype if dtype else type(value).__name__)
                            regular_keys.add(key)

                print(color_text(f"\nKeys in 'regular' and their count:", 'OKGREEN'))
                print(f"  Number of keys: {len(key_data_counts)}")
                for key, (count, dtype) in key_data_counts.items():
                    print(f"    {key}: {count} entries, Type: {dtype}")

                # 各キー (`dynamics`, `train`, `test`) の詳細を表示
                for key in ['dynamics', 'train', 'test']:
                    if key in regular_keys:
                        key_data_counts = {}

                        if key == 'dynamics':
                            for result in results:
                                if 'regular' in result and key in result['regular']:
                                    for entry in result['regular'][key]:
                                        if isinstance(entry, dict):
                                            for subkey, value in entry.items():
                                                if subkey not in key_data_counts:
                                                    key_data_counts[subkey] = (0, None)
                                                count, dtype = key_data_counts[subkey]
                                                key_data_counts[subkey] = (count + 1, dtype if dtype else type(value).__name__)
                        else:
                            for result in results:
                                if 'regular' in result and key in result['regular'] and isinstance(result['regular'][key], dict):
                                    for subkey, value in result['regular'][key].items():
                                        if subkey not in key_data_counts:
                                            key_data_counts[subkey] = (0, None)
                                        count, dtype = key_data_counts[subkey]
                                        key_data_counts[subkey] = (count + 1, dtype if dtype else type(value).__name__)

                        print(color_text(f"\nKeys in 'regular[{key}]' and their count:", 'OKGREEN'))
                        print(f"  Number of keys: {len(key_data_counts)}")
                        for subkey, (count, dtype) in key_data_counts.items():
                            print(f"    {subkey}: {count} entries, Type: {dtype}")

                # `dynamics` 内の `train` と `test` キーのサブキーを表示
                if 'dynamics' in regular_keys:
                    dynamics_train_keys = {}
                    dynamics_test_keys = {}
                    for result in results:
                        if 'regular' in result and 'dynamics' in result['regular']:
                            for entry in result['regular']['dynamics']:
                                if isinstance(entry, dict):
                                    if 'train' in entry and isinstance(entry['train'], dict):
                                        for subkey, value in entry['train'].items():
                                            if subkey not in dynamics_train_keys:
                                                dynamics_train_keys[subkey] = (0, None)
                                            count, dtype = dynamics_train_keys[subkey]
                                            dynamics_train_keys[subkey] = (count + 1, dtype if dtype else type(value).__name__)
                                    if 'test' in entry and isinstance(entry['test'], dict):
                                        for subkey, value in entry['test'].items():
                                            if subkey not in dynamics_test_keys:
                                                dynamics_test_keys[subkey] = (0, None)
                                            count, dtype = dynamics_test_keys[subkey]
                                            dynamics_test_keys[subkey] = (count + 1, dtype if dtype else type(value).__name__)

                    print(color_text(f"\nKeys in 'dynamics[train]' and their count:", 'OKGREEN'))
                    print(f"  Number of keys: {len(dynamics_train_keys)}")
                    for subkey, (count, dtype) in dynamics_train_keys.items():
                        print(f"    {subkey}: {count} entries, Type: {dtype}")

                    print(color_text(f"\nKeys in 'dynamics[test]' and their count:", 'OKGREEN'))
                    print(f"  Number of keys: {len(dynamics_test_keys)}")
                    for subkey, (count, dtype) in dynamics_test_keys.items():
                        print(f"    {subkey}: {count} entries, Type: {dtype}")

            # 詳細なデータを表示（verbose が True の場合のみ）
            if verbose:
                print(color_text("\nLoaded experiment results:", 'HEADER'))
                for i, result in enumerate(results):
                    print(color_text(f"\nResult {i+1}:", 'HEADER'))
                    if isinstance(result, dict):
                        print(f"  Type: dict")
                        print(f"  Keys: {list(result.keys())}")
                        print(f"  All Values:")
                        for key in result.keys():
                            print(f"    {key}: {result[key]}")
                    else:
                        print(f"  Type: {type(result).__name__}")
                        print(f"  Content: {result}")

    except Exception as e:
        print(f"An error occurred while loading the file: {e}")

# 使用例
file_path = 'F10k3Lsp_h_init/F10k3Lsp_h_init.pickle'
load_and_print_summary(file_path, verbose=True)  # 詳細表示を無効にする場合

Loaded HyperParams object:
  Type: HyperParams
  Attributes:
    L: 3 (int)
    alpha: 0.0001 (float)
    arch: fc_softplus (str)
    batch_seed: 0 (int)
    bias: False (bool)
    chunk: 100 (int)
    data_seed: 0 (int)
    delta_kernel: 0 (int)
    device: cuda (str)
    directory: F10k3Lsp_h_init (str)
    dtype: float64 (str)
    f0: 1 (int)
    final_kernel: 0 (int)
    h: 50 (int)
    init_kernel: 0 (int)
    init_seed: 0 (int)
    k: 3 (int)
    loss: softhinge (str)
    lossbeta: 20 (int)
    max_dgrad: 0.0001 (float)
    max_dout: 0.1 (float)
    n: 30 (int)
    normalize: False (bool)
    pickle: F10k3Lsp_h_init.pickle (str)
    regular: 1 (int)
    save_outputs: 0 (int)
    spbeta: 5 (int)
    store_kernel: 0 (int)
    tau_alpha_crit: 1000.0 (float)
    tau_over_h: 0.001 (float)
    test_size: 1000 (int)
    train_size: 700 (int)
    train_time: 18000 (int)

Experiment results summary:
  Number of results: 3
  Sample keys from result dictionaries:
    Number of keys: 3
    h